# SageMaker Coding Agent

A secure AI coding assistant powered by Amazon Bedrock Claude.

## Initialize Agent

In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from datetime import datetime

# Import agent components
from config import AgentConfig
from core.bedrock_client import BedrockClient
from core.tools import ToolRegistry, ToolContext
from core.agent_loop import AgentLoop
from core.security import SecurityManager, SecurityConfig
from core.audit import AuditLogger
from core.permissions import PermissionManager, PermissionResult
from core.memory import SessionManager
from core.context_manager import ContextManager
from core.project_config import ProjectConfig
from core.prompts import PromptBuilder

# Import tools
from tools import ALL_TOOLS

print("Components loaded successfully!")

In [ ]:
# Load configuration
config = AgentConfig.load()
print(f"Region: {config.region}")
print(f"Model: {config.primary_model}")

# Initialize components
workspace = os.path.abspath(config.workspace_root)

# Security
security_config = SecurityConfig(
    workspace_root=workspace,
    max_file_size=config.max_file_size,
    max_output_size=config.max_output_size,
    allow_network=config.allow_network,
)
security = SecurityManager(security_config)

# Audit logging
audit = AuditLogger(config.audit_dir)

# Session management
sessions = SessionManager(config.sessions_dir)

# Context management
context_mgr = ContextManager(workspace)

# Project config
project_config = ProjectConfig(workspace)

# Bedrock client
client = BedrockClient(config.primary_model, config.region)

# Tool registry
registry = ToolRegistry()
registry.register_all(ALL_TOOLS)

print(f"Workspace: {workspace}")
print(f"Tools registered: {len(registry.list_tools())}")
if project_config.has_instructions():
    print(f"Project instructions: {project_config.instruction_file}")

## Chat Interface

In [ ]:
# Global state
current_session = None
agent = None
pending_approval = None

# UI Components
chat_output = widgets.Output(layout=widgets.Layout(height='400px', overflow_y='auto'))
input_box = widgets.Textarea(
    placeholder='Type your message here...',
    layout=widgets.Layout(width='100%', height='80px')
)
send_button = widgets.Button(description='Send', button_style='primary')
clear_button = widgets.Button(description='Clear')
status_label = widgets.HTML(value='<b>Status:</b> Ready')

# Approval dialog
approval_output = widgets.Output()
approve_button = widgets.Button(description='Approve', button_style='success')
deny_button = widgets.Button(description='Deny', button_style='danger')
remember_checkbox = widgets.Checkbox(value=False, description='Remember for session')
approval_box = widgets.VBox([approval_output, widgets.HBox([approve_button, deny_button, remember_checkbox])])
approval_box.layout.display = 'none'

def add_message(role, content, tool_name=None):
    """Add message to chat display."""
    with chat_output:
        timestamp = datetime.now().strftime('%H:%M:%S')
        if role == 'user':
            display(HTML(f'<div style="background:#e3f2fd;padding:10px;margin:5px;border-radius:5px;"><b>[{timestamp}] You:</b><br>{content}</div>'))
        elif role == 'assistant':
            display(HTML(f'<div style="background:#f5f5f5;padding:10px;margin:5px;border-radius:5px;"><b>[{timestamp}] Agent:</b><br><pre style="white-space:pre-wrap;">{content}</pre></div>'))
        elif role == 'tool':
            display(HTML(f'<div style="background:#fff3e0;padding:10px;margin:5px;border-radius:5px;"><b>[{timestamp}] Tool ({tool_name}):</b><br><pre style="white-space:pre-wrap;font-size:12px;">{content[:500]}...</pre></div>' if len(content) > 500 else f'<div style="background:#fff3e0;padding:10px;margin:5px;border-radius:5px;"><b>[{timestamp}] Tool ({tool_name}):</b><br><pre style="white-space:pre-wrap;font-size:12px;">{content}</pre></div>'))
        elif role == 'system':
            display(HTML(f'<div style="background:#ffebee;padding:10px;margin:5px;border-radius:5px;"><b>[{timestamp}] System:</b> {content}</div>'))

def update_status(msg):
    """Update status label."""
    status_label.value = f'<b>Status:</b> {msg}'

def request_approval(tool_name, tool_input):
    """Show approval dialog."""
    global pending_approval
    pending_approval = {'tool': tool_name, 'input': tool_input, 'result': None}
    
    with approval_output:
        clear_output()
        display(HTML(f'<h4>Approval Required</h4>'))
        display(HTML(f'<p><b>Tool:</b> {tool_name}</p>'))
        display(HTML(f'<p><b>Input:</b></p><pre>{str(tool_input)[:500]}</pre>'))
    
    approval_box.layout.display = 'block'
    send_button.disabled = True
    
    # Wait for approval (blocking)
    import time
    while pending_approval['result'] is None:
        time.sleep(0.1)
    
    approval_box.layout.display = 'none'
    send_button.disabled = False
    
    return pending_approval['result']

def on_approve(b):
    global pending_approval
    if pending_approval:
        pending_approval['result'] = True
        add_message('system', f"Approved: {pending_approval['tool']}")

def on_deny(b):
    global pending_approval
    if pending_approval:
        pending_approval['result'] = False
        add_message('system', f"Denied: {pending_approval['tool']}")

approve_button.on_click(on_approve)
deny_button.on_click(on_deny)

def on_send(b):
    """Handle send button click."""
    global current_session, agent
    
    message = input_box.value.strip()
    if not message:
        return
    
    input_box.value = ''
    add_message('user', message)
    update_status('Processing...')
    
    # Create session if needed
    if current_session is None:
        current_session = sessions.create(message[:50])
    
    # Create tool context
    ctx = ToolContext(
        working_dir=workspace,
        session_id=current_session.id,
        security_manager=security,
        audit_logger=audit,
    )
    
    # Build system prompt
    prompt_builder = PromptBuilder()
    system_prompt = prompt_builder.build(
        workspace_root=workspace,
        project_instructions=project_config.get_instructions(),
    )
    
    # Create agent
    agent = AgentLoop(
        client=client,
        registry=registry,
        system_prompt=system_prompt,
        context=ctx,
        max_turns=config.max_turns,
        doom_threshold=config.doom_loop_threshold,
        on_text=lambda t: add_message('assistant', t),
        on_tool_call=lambda n, i: update_status(f'Calling {n}...'),
        on_tool_result=lambda n, r: add_message('tool', r, n),
        on_approval=request_approval,
    )
    
    try:
        response = agent.run(message)
        
        # Save to session
        sessions.add_message(current_session, 'user', message)
        sessions.add_message(current_session, 'assistant', response)
        
        # Check context
        warning = context_mgr.check_and_warn(agent.get_messages())
        if warning:
            add_message('system', warning)
        
        update_status('Ready')
        
    except Exception as e:
        add_message('system', f'Error: {str(e)}')
        update_status('Error')

def on_clear(b):
    """Clear chat and start new session."""
    global current_session, agent
    current_session = None
    agent = None
    chat_output.clear_output()
    update_status('Ready')
    with chat_output:
        display(HTML('<p><i>Chat cleared. Start a new conversation.</i></p>'))

send_button.on_click(on_send)
clear_button.on_click(on_clear)

# Layout
header = widgets.HTML('<h2>SageMaker Coding Agent</h2>')
button_row = widgets.HBox([send_button, clear_button, status_label])
chat_box = widgets.VBox([header, chat_output, approval_box, input_box, button_row])

display(chat_box)

## Session Management

In [ ]:
# List recent sessions
recent = sessions.list_sessions()[:5]
if recent:
    print("Recent sessions:")
    for s in recent:
        print(f"  - {s['id']}: {s['title']} ({s['message_count']} messages)")
else:
    print("No previous sessions.")

In [ ]:
# Load a previous session (uncomment and set session_id)
# session_id = "20240101_120000"
# current_session = sessions.load(session_id)
# if current_session:
#     print(f"Loaded session: {current_session.title}")
#     for msg in current_session.messages[-5:]:
#         print(f"  [{msg['role']}]: {str(msg['content'])[:100]}...")

## Context Status

In [ ]:
# Check context status
if agent:
    status = context_mgr.get_status(agent.get_messages())
    print(f"Context usage: {status['usage_percent']:.1%}")
    print(f"Tokens: {status['tokens']:,} / {status['max_tokens']:,}")
    print(f"Level: {status['warning_level']}")
else:
    print("No active conversation.")

## Audit Log

In [ ]:
# View audit log for current session
if current_session:
    entries = audit.get_session_log(current_session.id)
    print(f"Audit entries for session {current_session.id}: {len(entries)}")
    for e in entries[-5:]:
        print(f"  [{e['timestamp']}] {e['action']}: {e['tool_name']} - {e['result_summary'][:50]}")
else:
    print("No active session.")